# La Transformada de Laplace

## Introducción

Sympy permite calcular transformadas de Laplace mediante la función **laplace_transform**, y la correspondiente transformada inversa mediante la función **inverse_laplace_transform**. En el caso de la transformada directa de manera predeterminada **laplace_transform** realiza el cálculo de la transformada y nos devuelve la función, la abscisa de convergencia y condiciones de convergencia auxiliares. En la mayoría de las situaciones prácticas nos interesa únicamente obtener el cálculo de la transformada de la función, por lo vamos a definir nuestras propias funciones para el cálculo de la transformada directa y la transformada inversa

In [18]:
import sympy as sp

# Se define la Transformada de Laplace
def laplace(f):
    s = sp.symbols('s')
    t = sp.symbols ('t',real=True)
    return sp.laplace_transform(f, t, s, noconds=True)
# Se define la Transformada de Laplace inversa
def ilaplace(F):
    s = sp.symbols('s')
    t = sp.symbols ('t',real=True)
    return sp.inverse_laplace_transform(F, s, t, noconds=True)

Ahora definimos las variables simbólicas que vamos a utilizar para trabajar

In [7]:
t = sp.symbols("t",real=True)
s = sp.symbols("s")

A modo de ejemplo, consideraremos la función $f(t)=t^2+\cos(3t)$

In [8]:
f_t = sp.cos(3*t)+t**2
f_t

t**2 + cos(3*t)

Y calculamos $F(s)=\mathscr{L}\{f(t)\}$

In [12]:
F_s = laplace(f_t)
F_s

s/(s**2 + 9) + 2/s**3

Podemos recuperar $f(t)$ mediante la transformada inversa $\mathscr{L}^{-1}\{f(t)\}$

In [20]:
ilaplace(F_s)

t**2*Heaviside(t) + cos(3*t)*Heaviside(t)

## Escalón unitario de Heaviside

Al utilizar la función **ilaplace(F_s)** obtuvimos nuestra función $f(t)$ multiplicada por otra función $θ(t)$. Esta última es la manera en que Sympy expresa al escalón unitario de Heaviside:

In [23]:
Heaviside = sp.Heaviside(t)
Heaviside

Heaviside(t)

Un detalle importante a tener en cuenta es que la definición del escalón unitario de Heaviside implementado por Sympy difiere a la definición adoptada en nuestra materia. La implementación de Sympy considera:


$$\theta(t)=\begin{cases}
0, \quad t<0,\\
\frac{1}{2}, \quad t=0,\\
1,  \quad t>0.
\end{cases}
$$


Esto lo podemos comprobar evaluando la función en $t=0$

In [25]:
Heaviside.subs(t,0)

1/2

## Ejemplos

En situaciones donde tengamos cocientes de polinomios sencillos de la forma

In [75]:
F_s2 = (1/(s+3))*(1/(s+1))
F_s2

1/((s + 1)*(s + 3))

Podemos encontrar la transformada inversa de manera directa

In [45]:
ilaplace(F_s2)

exp(-t)*Heaviside(t)/2 - exp(-3*t)*Heaviside(t)/2

Ahora en casos donde el cociente de polinomios tenga una estructura mas compleja, de la forma

In [76]:
F_s3 = (s**4+2*s**2+18)/(s**3*(s**2+9))
F_s3

(s**4 + 2*s**2 + 18)/(s**3*(s**2 + 9))

Es una buena práctica realizar la descomposición en fracciones simples antes de realizar el cálculo de la transformada inversa. Esto podemos hacerlo mediante el método **apart(s)**

In [77]:
F_s3 = F_s3.apart(s)
F_s3

s/(s**2 + 9) + 2/s**3

Y sobre esta nueva expresión podemos calcular la transformada inversa

In [78]:
ilaplace(F_s3)

t**2*Heaviside(t) + cos(3*t)*Heaviside(t)

Es importante tener en cuenta que si nuestra $F(s)$ no es exclusivamente un cociente de polinomios, el método **apart(s)** no puede aplicarse directamente. Este es el caso en que tengamos exponenciales complejas en el dominio $s$

In [82]:
F_s3 = sp.exp(-s)*(s**4+2*s**2+18)/(s**3*(s**2+9))
F_s3

(s**4 + 2*s**2 + 18)*exp(-s)/(s**3*(s**2 + 9))

Si ahora intentamos expandir $F(s)$ en fracciones simples obtenemos un error

In [81]:
F_s3.apart(s)

PolynomialError: exp(s) contains an element of the set of generators.

En este tipo de situaciones podemos intentar calcular directamente la transformada inversa sobre la función

In [84]:
ilaplace(F_s3)

(t - 1)**2*Heaviside(t - 1) + cos(3*t - 3)*Heaviside(t - 1)

Vemos que en este caso el software fue capaz de llegar a una solución. En situaciones donde este no sea capaz de arribar a un resultado podemos valernos de nuestros conocimientos teóricos para resolver el problema, por ejemplo, podemos hacer uso del segundo teorema de la traslación

In [93]:
# Definimos una nueva función F(s) sin la exponencial compleja
F_s3_ = (s**4+2*s**2+18)/(s**3*(s**2+9))
F_s3_

(s**4 + 2*s**2 + 18)/(s**3*(s**2 + 9))

In [94]:
# Calculamos su transformada inversa
f_t3_aux = ilaplace(F_s3_)
f_t3_aux

t**2*Heaviside(t) + cos(3*t)*Heaviside(t)

In [95]:
# Aplicamos el corrimiento temporal sobre la función encontrada
f_t3_aux.subs(t,t-1)

(t - 1)**2*Heaviside(t - 1) + cos(3*t - 3)*Heaviside(t - 1)

## Consideraciones a tener en cuenta

A pesar de todas sus funcionalidades, Sympy aún no es una librería robusta en el cálculo de transformadas de Laplace, especialmente en el cálculo de transformadas inversas. La capacidad de cálculo de la antitransformada generalmente depende en gran medida de **cómo** expresamos nuestra función de interés sobre la cual queremos aplicar el operador. Uno de los motivos comunes por los que el software no es capaz de realizar el cálculo de una transformada inversa se debe a que las constantes numéricas involucradas están expresadas como valores flotantes y no como variables simbólicas. Por ejemplo, si una constante involucrada en el cálculo es $k = 0.5$ y el software no es capaz de resolver la antitransformada, una posible solución al problema es expresar la misma como una variable simbólica de la forma 

In [99]:
k = sp.Rational(1,2)
k


1/2